In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

In [2]:
examples = [
    { "movie": "Top Gun", "answer": "🛩️👨‍✈️🔥" },
    { "movie": "The Godfather", "answer": "👨‍👨‍👦🔫🍝" },
    { "movie": "Finding Nemo", "answer": "🐟🛡️🌊" },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What are three emojis for '{movie}'?"),
    ("ai", "{answer}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI that provides three emojis representing a movie title."),
    few_shot_prompt,
    ("system", "Now conversation started."),
    ("human", "{question}"),
])

In [3]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

In [4]:
invoke_chain("What are three emojis for 'Inception'?")
invoke_chain("What are three emojis for 'Frozen'?")
invoke_chain("What was the first movie I asked about?")

🧠🕰️🌌
❄️👸⛄
The first movie you asked about was 'Top Gun'.
